In [ ]:
def fix_incomplete_json(string) -> dict:
    
    # Assuming json is truncated thus rendered incomplete

    pairs = {
        '[': ']',
        '{': '}',
        '"': '"'
    }
    open_stack = []
    dict_suffix = ':0'

    string = string.replace("\'", "\"")

    for char in string:
        if char == '"':
            prev_open = open_stack[-1:][0]
            if prev_open != char:
                open_stack.append(char)
            else:
                open_stack.pop()

        elif char in pairs.keys():
            open_stack.append(char)

        elif char in pairs.values():
            prev_open = open_stack.pop()
            if pairs[prev_open] != char:
                print(f"char: {char}, prev_open: {prev_open}")
                raise Exception('invalid')

    open_stack.reverse()
    to_append = ''.join([pairs[open_char] for open_char in open_stack])
    string += to_append

    try:
        data = json.loads(string)
    except ValueError as e:
        msg = str(e)
        delimiter_pat = r"(?<=Expecting ')."
        char_index_pat = r'(?<=char )\d+'

        ind = int(re.search(char_index_pat, msg)[0])
        delim = re.search(delimiter_pat, msg)[0]

        if delim == ':':
            string = string[:ind] + dict_suffix + string[ind:]

        data = json.loads(string)

    return data